<a href="https://colab.research.google.com/github/Cralsic123/Detr-resnet50-metrics-comparison/blob/main/Detr_resnet50_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision
!pip install grad-cam
!git clone https://github.com/quic/ai-hub-models.git

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:

import torch
import torchvision.transforms as transforms
from torchvision import models
#from gradcam import GradCAM, show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


## DETR model


In [5]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection


Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
  0% 0.00/15.1M [00:00<?, ?B/s]
100% 15.1M/15.1M [00:00<00:00, 232MB/s]


In [7]:
import zipfile
with zipfile.ZipFile('/content/brain-mri-images-for-brain-tumor-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


In [8]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [22]:
image_path = '/content/brain_tumor_dataset/yes/Y1.jpg'
image = Image.open(image_path)

In [23]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [24]:
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]


In [25]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected cup with confidence 0.9 at location [18.09, 72.82, 90.13, 190.65]


In [ ]:
#model.eval()

In [26]:
import os

In [14]:
folder_path = '/content/brain_tumor_dataset/yes'

# List all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'JPG'))]


for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = Image.open(image_path)

    # Preprocess the image
    inputs = processor(images=image, return_tensors="pt")
    
    # Forward pass through the model
    outputs = model(**inputs)

    # Post-process the results
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    # Print the results for the current image
    print(f"Results for {image_file}:")
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        print(
            f"  Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
        )
    print()

## Grad Cam visualization

In [27]:
import warnings
warnings.filterwarnings('ignore')
from torchvision import transforms
#from datasets import load_dataset
from pytorch_grad_cam import run_dff_on_image, GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import cv2
import torch
from typing import List, Callable, Optional

In [ ]:
img_tensor = transforms.ToTensor()(image)

In [ ]:
# class HuggingfaceToTensorModelWrapper(torch.nn.Module):
#     def __init__(self, model):
#         super(HuggingfaceToTensorModelWrapper, self).__init__()
#         self.model = model

#     def forward(self, x):
#         return self.model(x).logits

# """ Translate the category name to the category index.
#     Some models aren't trained on Imagenet but on even larger datasets,
#     so we can't just assume that 761 will always be remote-control.

# """
# def category_name_to_index(model, category_name):
#     name_to_index = dict((v, k) for k, v in model.config.id2label.items())
#     return name_to_index[category_name]

# """ Helper function to run GradCAM on an image and create a visualization.
#     (note to myself: this is probably useful enough to move into the package)
#     If several targets are passed in targets_for_gradcam,
#     e.g different categories,
#     a visualization for each of them will be created.

# """
# def run_grad_cam_on_image(model: torch.nn.Module,
#                           target_layer: torch.nn.Module,
#                           targets_for_gradcam: List[Callable],
#                           reshape_transform: Optional[Callable],
#                           input_tensor: torch.nn.Module=img_tensor,
#                           input_image: Image=image,
#                           method: Callable=GradCAM):
#     with method(model=HuggingfaceToTensorModelWrapper(model),
#                  target_layers=[target_layer],
#                  reshape_transform=reshape_transform) as cam:

#         # Replicate the tensor for each of the categories we want to create Grad-CAM for:
#         repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

#         batch_results = cam(input_tensor=repeated_tensor,
#                             targets=targets_for_gradcam)
#         results = []
#         for grayscale_cam in batch_results:
#             visualization = show_cam_on_image(np.float32(input_image)/255,
#                                               grayscale_cam,
#                                               use_rgb=True)
#             # Make it weight less in the notebook:
#             visualization = cv2.resize(visualization,
#                                        (visualization.shape[1]//2, visualization.shape[0]//2))
#             results.append(visualization)
#         return np.hstack(results)


# def print_top_categories(model, img_tensor, top_k=5):
#     logits = model(img_tensor.unsqueeze(0)).logits
#     indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
#     for i in indices:
#         print(f"Predicted class {i}: {model.config.id2label[i]}")

In [ ]:

# We will show GradCAM for the "Egyptian Cat" and the 'Remote Control" categories:
# I changed it to cup because the model is detecting the anomaly in the brain as a cup
targets_for_gradcam = [ClassifierOutputTarget(category_name_to_index(model, "cup"))]

In [ ]:
#print(model)

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): ResNetBackbone(
          (embedder): ResNetEmbeddings(
            (embedder): ResNetConvLayer(
              (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
              (normalization): DetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          )
          (encoder): ResNetEncoder(
            (stages): ModuleList(
              (0): ResNetStage(
                (layers): Sequential(
                  (0): ResNetBottleNeckLayer(
                    (shortcut): ResNetShortCut(
                      (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                      (normalization): DetrFrozenBatchNorm2d()
                    )
                    (layer): Seq

In [ ]:
# target_layer = model.model.decoder.layernorm
# classifier = model.class_labels_classifier

In [ ]:
# The last layer in the Resnet Encoder:
#target_layer = model.encoder.stages[-1].layers[-1]
#target_layer = model.ResNetEncoder.stages[-1].layers[-1]
#target_layer = model.model.backbone.conv_encoder.model.encoder.stages[-1].layers[-1]
model.model.decoder.layernorm

display(Image.fromarray(run_dff_on_image(model=model,
                          target_layer=target_layer,
                          classifier=classifier,
                          img_pil=image,
                          img_tensor=img_tensor,
                          reshape_transform=None,
                          n_components=4,
                          top_k=2)))
display(Image.fromarray(run_grad_cam_on_image(model=model,
                      target_layer=target_layer,
                      targets_for_gradcam=targets_for_gradcam,
                      reshape_transform=None)))
print_top_categories(model, img_tensor)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x2048 and 256x92)

The error RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x2048 and 256x92) indicates that there's a mismatch between the shapes of the matrices being multiplied, likely due to the wrong layer being used as target_layer. The layernorm layer has an output of shape (batch_size, sequence_length, 256), which is not directly compatible with the classifier expecting an input of shape (batch_size, 256).

To solve this, we should use the correct layer's output, which matches the input expected by the classifier. The target_layer should be the last layer in the decoder before the classifier. In the DETR model, this is typically the output of the decoder.

In [ ]:
# Displaying DFF (Dimensionality Filter Framework) output
dff_image = run_dff_on_image(
    model=model,
    target_layer=target_layer,
    classifier=classifier,
    img_pil=image,
    img_tensor=img_tensor,
    reshape_transform=None,
    n_components=4,
    top_k=2
)
display(Image.fromarray(dff_image))

# Displaying Grad-CAM output
grad_cam_image = run_grad_cam_on_image(
    model=model,
    target_layer=target_layer,
    targets_for_gradcam=targets_for_gradcam,
    reshape_transform=None
)
display(Image.fromarray(grad_cam_image))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x2048 and 256x92)

In [ ]:
from PIL import Image
import torch

# Let's define a helper function to find the correct target layer
def get_target_layer_and_output(model, img_tensor):
    # Forward pass to find the correct layer output
    outputs = model.model.backbone(img_tensor)
    target_layer_output = outputs  # Adjust this according to actual layer outputs
    return model.model.backbone, target_layer_output

# Get the target layer and output
target_layer, target_layer_output = get_target_layer_and_output(model, img_tensor)

# Displaying DFF (Dimensionality Filter Framework) output
try:
    dff_image = run_dff_on_image(
        model=model,
        target_layer=target_layer,
        classifier=classifier,
        img_pil=image,
        img_tensor=img_tensor,
        reshape_transform=None,
        n_components=4,
        top_k=2
    )
    display(Image.fromarray(dff_image))
except RuntimeError as e:
    print(f"RuntimeError during DFF: {e}")

# Displaying Grad-CAM output
try:
    grad_cam_image = run_grad_cam_on_image(
        model=model,
        target_layer=target_layer,
        targets_for_gradcam=targets_for_gradcam,
        reshape_transform=None
    )
    display(Image.fromarray(grad_cam_image))
except RuntimeError as e:
    print(f"RuntimeError during Grad-CAM: {e}")

# Printing the top categories
try:
    print_top_categories(model, img_tensor)
except RuntimeError as e:
    print(f"RuntimeError during category printing: {e}")


TypeError: DetrConvModel.forward() missing 1 required positional argument: 'pixel_mask'